In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/preprocessed-dataset/test_up.csv
/kaggle/input/preprocessed-dataset/train_up.csv
/kaggle/input/pfinal/fianl2_trainy.csv
/kaggle/input/pfinal/final1_trainX.csv
/kaggle/input/pfinal/final3_test.csv


In [10]:
X = pd.read_csv("/kaggle/input/pfinal/final1_trainX.csv")
y = pd.read_csv("/kaggle/input/pfinal/fianl2_trainy.csv")
X_pred = pd.read_csv("/kaggle/input/pfinal/final3_test.csv")

In [11]:
X = X.drop(['Unnamed: 0'], axis=1)
y = y.drop(['Unnamed: 0'], axis=1)
X_pred = X_pred.drop(['Unnamed: 0'], axis=1)

# **LINEAR REGRESSION**

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state=50)

regr = LinearRegression()

regr.fit(X_train, y_train)
print(regr.score(X_test, y_test))

0.8492585673125327


In [5]:
pred=regr.predict(X_test)

In [6]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, pred)

0.013414499341271405

# **RANDOM FOREST**

In [18]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestRegressor()

In [19]:
pred=rf.predict(X_test)

In [20]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, pred)

0.007950071550140518

# **XGBOOST**

In [10]:
from xgboost import XGBRegressor
# tree_method='gpu_hist'
xgbr = XGBRegressor()
xgbr.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [11]:
score = xgbr.score(X_train, y_train)
print("Training Score: ", score)

Training Score:  0.9308332849828457


In [12]:
y_pred = xgbr.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("MSE: ", mse)

MSE:  0.006219837244864773


# XGBR with different hyperparameters

In [13]:
params={
  "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
  "max_depth"        : [ 100, 130, 180, 220, 264],
  "min_child_weight" : [ 1, 3, 5, 7 ],
  "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
  "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
 }

In [14]:
xgbr = XGBRegressor(xgbr = XGBRegressor(tree_method='gpu_hist',  min_child_weight = 5,
 max_depth = 130,
 learning_rate = 0.008,
 gamma = 0.3,
 colsample_bytree = 0.7))

In [17]:
xgbr.fit(X_train, y_train)
score = xgbr.score(X_train, y_train)
print("Training Score: ", score)

[16:42:39] WARNING: ../src/learner.cc:627: 
Parameters: { "xgbr" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


Training Score:  0.9252994387097826


In [18]:
from sklearn.metrics import mean_squared_error
# prediction
y_pred = xgbr.predict(X_test)

# accuracy check
mean_squared_error(y_test, y_pred)

0.007041803256523976

# Random Forest Hyperparameter Tuning

# Model 1

In [2]:
X = pd.read_csv("/kaggle/input/pfinal/final1_trainX.csv")
y = pd.read_csv("/kaggle/input/pfinal/fianl2_trainy.csv")
test = pd.read_csv("/kaggle/input/pfinal/final3_test.csv")

****Here we are going to do manually because we don't have the computing power to do this with grid serach cv or random serach cv****

In [3]:
X = X.drop(['Unnamed: 0','matchType'], axis=1)
y = y.drop(['Unnamed: 0'], axis=1)
test = test.drop(['Unnamed: 0','matchType'], axis=1)

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=50)

In [5]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=40, min_samples_leaf=3, max_features='sqrt',
                          n_jobs=-1)
rf.fit(X_train, y_train)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


RandomForestRegressor(max_features='sqrt', min_samples_leaf=3, n_estimators=40,
                      n_jobs=-1)

In [7]:
pred=rf.predict(X_test)

In [8]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, pred)

0.00785421116171999

# Model 2

In [10]:
rf = RandomForestRegressor(n_estimators=80, min_samples_leaf=4, max_features='sqrt',
                          n_jobs=-1)
rf.fit(X_train, y_train)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestRegressor(max_features='sqrt', min_samples_leaf=4, n_estimators=80,
                      n_jobs=-1)

In [11]:
pred=rf.predict(X_test)

In [12]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, pred)

0.007789099707292755

# Model 3

In [13]:
rf = RandomForestRegressor(n_estimators=120, min_samples_leaf=3, max_features='sqrt',
                          n_jobs=-1)
rf.fit(X_train, y_train)
pred=rf.predict(X_test)
mean_squared_error(y_test, pred)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


0.007786983146649776

# Model 4

In [14]:
rf = RandomForestRegressor(n_estimators=160, min_samples_leaf=4, max_features='sqrt',
                          n_jobs=-1)
rf.fit(X_train, y_train)
pred=rf.predict(X_test)
mean_squared_error(y_test, pred)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


0.007747772399509248

# Model 5

In [15]:
rf = RandomForestRegressor(n_estimators=200, min_samples_leaf=4, max_features='sqrt',
                          n_jobs=-1)
rf.fit(X_train, y_train)
pred=rf.predict(X_test)
mean_squared_error(y_test, pred)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


0.007751419173244444

# Model 6

In [16]:
rf = RandomForestRegressor(n_estimators=180, min_samples_leaf=3, max_features='log2',
                          n_jobs=-1)
rf.fit(X_train, y_train)
pred=rf.predict(X_test)
mean_squared_error(y_test, pred)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


0.0077561377723093464

# Model 7

In [17]:
rf = RandomForestRegressor(n_estimators=150, min_samples_leaf=4, max_features='log2',
                          n_jobs=-1)
rf.fit(X_train, y_train)
pred=rf.predict(X_test)
mean_squared_error(y_test, pred)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


0.007754615270349284